# Model Training
ANN,LSTM,GRU models are trained

In [78]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
import tensorflow as tf
import math
from zipfile import ZipFile
import pickle
import os
import numpy as np
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import joblib 

In [6]:
stock_tickers = ['AAPL','GOOG','TSLA','MSFT']

In [41]:
#read in the files
apple = pd.read_csv('../data/AAPL.csv')
google = pd.read_csv('../data/GOOG.csv')
tesla = pd.read_csv('../data/TSLA.csv')
microsoft = pd.read_csv('../data/MSFT.csv')

In [42]:
#format the dataframes
apple = apple.drop(apple.columns[0],1)
google = google.drop(google.columns[0],1)
tesla = tesla.drop(tesla.columns[0],1)
microsoft = microsoft.drop(microsoft.columns[0],1)

/var/folders/8y/6j0y5pg92ng64xbpn5hywscc0000gn/T/ipykernel_31939/1336139938.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  apple = apple.drop(apple.columns[0],1)
/var/folders/8y/6j0y5pg92ng64xbpn5hywscc0000gn/T/ipykernel_31939/1336139938.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  google = google.drop(google.columns[0],1)
/var/folders/8y/6j0y5pg92ng64xbpn5hywscc0000gn/T/ipykernel_31939/1336139938.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  tesla = tesla.drop(tesla.columns[0],1)
/var/folders/8y/6j0y5pg92ng64xbpn5hywscc0000gn/T/ipykernel_31939/1336139938.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  microsoft = micr

In [ ]:
#features to be used
indicators = ['High','Low','Open','Volume','Adj Close','H-L','O-C','5MA',
              '10MA','20MA','7SD','EMA8','EMA21','EMA34','EMA55','RSI_14','Sentiment']

# Training ANN model for AAPL

In [75]:
X = np.asarray(apple[indicators], np.float32)
Y = np.asarray(apple['Close'], np.float32)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

y_train = y_train.reshape(-1,1)
mms_X = MinMaxScaler()
mms_y = MinMaxScaler()

scaler_x = mms_X.fit(X_train)
scaler_y = mms_y.fit(y_train)

X_train = scaler_x.transform(X_train)
y_train = scaler_y.transform(y_train)

X_test = scaler_x.transform(X_test)
y_test = y_test.reshape(-1,1)
y_test = scaler_y.transform(y_test)

pca = PCA(n_components=4)

X_train= pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# Defining ANN neural network
model = Sequential()
model.add(Dense(units=50, input_dim=4, kernel_initializer='normal', activation='relu'))
model.add(Dense(units=25, kernel_initializer='normal', activation='tanh'))
model.add(Dense(units=10, kernel_initializer='normal', activation='tanh'))
model.add(Dense(1, kernel_initializer='normal'))

# Compiling the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Fitting the ANN to the Training set
history = model.fit(X_train, y_train ,batch_size = 10, validation_data = (X_test, y_test), epochs = 15, verbose=1)

#save the model
model.save(r'../models/AAPL_ann.h5')

#save the feature data normalizer
scalerfilename_x = f'../normalizers/scaler_x_AAPL_ann.pkl'
pickle.dump(scaler_x, open(scalerfilename_x, 'wb'))


# #save the output normaliser
scalerfilename_y = f'../normalizers/scaler_y_AAPL_ann.pkl'
pickle.dump(scaler_y, open(scalerfilename_y, 'wb'))

Epoch 1/15
8/8 [==============================] - ETA: 0s - loss: 0.2790

2022-07-15 03:05:47.906091: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-15 03:05:48.092190: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


8/8 [==============================] - 1s 19ms/step - loss: 0.2790 - val_loss: 0.2815
Epoch 2/15
8/8 [==============================] - 0s 7ms/step - loss: 0.2666 - val_loss: 0.2650
Epoch 3/15
8/8 [==============================] - 0s 6ms/step - loss: 0.2467 - val_loss: 0.2395
Epoch 4/15
8/8 [==============================] - 0s 6ms/step - loss: 0.2142 - val_loss: 0.1992
Epoch 5/15
8/8 [==============================] - 0s 6ms/step - loss: 0.1680 - val_loss: 0.1419
Epoch 6/15
8/8 [==============================] - 0s 6ms/step - loss: 0.1085 - val_loss: 0.0814
Epoch 7/15
8/8 [==============================] - 0s 6ms/step - loss: 0.0560 - val_loss: 0.0403
Epoch 8/15
8/8 [==============================] - 0s 6ms/step - loss: 0.0236 - val_loss: 0.0250
Epoch 9/15
8/8 [==============================] - 0s 6ms/step - loss: 0.0149 - val_loss: 0.0172
Epoch 10/15
8/8 [==============================] - 0s 6ms/step - loss: 0.0100 - val_loss: 0.0110
Epoch 11/15
8/8 [==============================] 

# Training ANN model for TSLA

In [53]:
X = np.asarray(tesla[indicators], np.float32)
Y = np.asarray(tesla['Close'], np.float32)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

y_train = y_train.reshape(-1,1)
mms_X = MinMaxScaler()
mms_y = MinMaxScaler()

scaler_x = mms_X.fit(X_train)
scaler_y = mms_y.fit(y_train)

X_train = scaler_x.transform(X_train)
y_train = scaler_y.transform(y_train)

X_test = scaler_x.transform(X_test)
y_test = y_test.reshape(-1,1)
y_test = scaler_y.transform(y_test)

pca = PCA(n_components=4)

X_train= pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# Defining ANN neural network
model = Sequential()
model.add(Dense(units=50, input_dim=4, kernel_initializer='normal', activation='relu'))
model.add(Dense(units=25, kernel_initializer='normal', activation='tanh'))
model.add(Dense(units=10, kernel_initializer='normal', activation='tanh'))
model.add(Dense(1, kernel_initializer='normal'))

# Compiling the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Fitting the ANN to the Training set
history = model.fit(X_train, y_train ,batch_size = 10, validation_data = (X_test, y_test), epochs = 15, verbose=1)

#save the model
model.save(r'../models/TSLA_ann.h5')

#save the feature data normalizer
scalerfilename_x = f'../normalizers/scaler_x_TSLA_ann.pkl'
pickle.dump(scaler_x, open(scalerfilename_x, 'wb'))


# #save the output normaliser
scalerfilename_y = f'../normalizers/scaler_y_TSLA_ann.pkl'
pickle.dump(scaler_y, open(scalerfilename_y, 'wb'))

Epoch 1/15
8/8 [==============================] - ETA: 0s - loss: 0.3033

2022-07-15 02:19:48.406521: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-15 02:19:48.587278: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


8/8 [==============================] - 0s 17ms/step - loss: 0.3033 - val_loss: 0.2460
Epoch 2/15
8/8 [==============================] - 0s 6ms/step - loss: 0.2848 - val_loss: 0.2245
Epoch 3/15
8/8 [==============================] - 0s 6ms/step - loss: 0.2558 - val_loss: 0.1936
Epoch 4/15
8/8 [==============================] - 0s 6ms/step - loss: 0.2153 - val_loss: 0.1486
Epoch 5/15
8/8 [==============================] - 0s 6ms/step - loss: 0.1557 - val_loss: 0.0915
Epoch 6/15
8/8 [==============================] - 0s 7ms/step - loss: 0.0836 - val_loss: 0.0406
Epoch 7/15
8/8 [==============================] - 0s 6ms/step - loss: 0.0326 - val_loss: 0.0200
Epoch 8/15
8/8 [==============================] - 0s 6ms/step - loss: 0.0172 - val_loss: 0.0176
Epoch 9/15
8/8 [==============================] - 0s 6ms/step - loss: 0.0131 - val_loss: 0.0102
Epoch 10/15
8/8 [==============================] - 0s 6ms/step - loss: 0.0067 - val_loss: 0.0051
Epoch 11/15
8/8 [==============================] 

# Training ANN model for GOOG

In [54]:
X = np.asarray(google[indicators], np.float32)
Y = np.asarray(google['Close'], np.float32)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

y_train = y_train.reshape(-1,1)
mms_X = MinMaxScaler()
mms_y = MinMaxScaler()

scaler_x = mms_X.fit(X_train)
scaler_y = mms_y.fit(y_train)

X_train = scaler_x.transform(X_train)
y_train = scaler_y.transform(y_train)

X_test = scaler_x.transform(X_test)
y_test = y_test.reshape(-1,1)
y_test = scaler_y.transform(y_test)

pca = PCA(n_components=4)

X_train= pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# Defining ANN neural network
model = Sequential()
model.add(Dense(units=50, input_dim=4, kernel_initializer='normal', activation='relu'))
model.add(Dense(units=25, kernel_initializer='normal', activation='tanh'))
model.add(Dense(units=10, kernel_initializer='normal', activation='tanh'))
model.add(Dense(1, kernel_initializer='normal'))

# Compiling the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Fitting the ANN to the Training set
history = model.fit(X_train, y_train ,batch_size = 10, validation_data = (X_test, y_test), epochs = 15, verbose=1)

#save the model
model.save(r'../models/GOOG_ann.h5')

#save the feature data normalizer
scalerfilename_x = f'../normalizers/scaler_x_GOOG_ann.pkl'
pickle.dump(scaler_x, open(scalerfilename_x, 'wb'))


# #save the output normaliser
scalerfilename_y = f'../normalizers/scaler_y_GOOG_ann.pkl'
pickle.dump(scaler_y, open(scalerfilename_y, 'wb'))

Epoch 1/15
8/8 [==============================] - 1s 17ms/step - loss: 0.4042 - val_loss: 0.3406
Epoch 2/15
1/8 [==>...........................] - ETA: 0s - loss: 0.3635

2022-07-15 02:20:57.752074: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-15 02:20:57.921435: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


8/8 [==============================] - 0s 6ms/step - loss: 0.3809 - val_loss: 0.3154
Epoch 3/15
8/8 [==============================] - 0s 6ms/step - loss: 0.3492 - val_loss: 0.2778
Epoch 4/15
8/8 [==============================] - 0s 6ms/step - loss: 0.2983 - val_loss: 0.2222
Epoch 5/15
8/8 [==============================] - 0s 6ms/step - loss: 0.2271 - val_loss: 0.1481
Epoch 6/15
8/8 [==============================] - 0s 7ms/step - loss: 0.1416 - val_loss: 0.0748
Epoch 7/15
8/8 [==============================] - 0s 6ms/step - loss: 0.0740 - val_loss: 0.0386
Epoch 8/15
8/8 [==============================] - 0s 6ms/step - loss: 0.0418 - val_loss: 0.0296
Epoch 9/15
8/8 [==============================] - 0s 6ms/step - loss: 0.0297 - val_loss: 0.0185
Epoch 10/15
8/8 [==============================] - 0s 6ms/step - loss: 0.0186 - val_loss: 0.0097
Epoch 11/15
8/8 [==============================] - 0s 6ms/step - loss: 0.0132 - val_loss: 0.0072
Epoch 12/15
8/8 [==============================] 

# Training ANN model for MSFT

In [56]:
X = np.asarray(microsoft[indicators], np.float32)
Y = np.asarray(microsoft['Close'], np.float32)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

y_train = y_train.reshape(-1,1)
mms_X = MinMaxScaler()
mms_y = MinMaxScaler()

scaler_x = mms_X.fit(X_train)
scaler_y = mms_y.fit(y_train)

X_train = scaler_x.transform(X_train)
y_train = scaler_y.transform(y_train)

X_test = scaler_x.transform(X_test)
y_test = y_test.reshape(-1,1)
y_test = scaler_y.transform(y_test)

pca = PCA(n_components=4)

X_train= pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# Defining ANN neural network
model = Sequential()
model.add(Dense(units=50, input_dim=4, kernel_initializer='normal', activation='relu'))
model.add(Dense(units=25, kernel_initializer='normal', activation='tanh'))
model.add(Dense(units=10, kernel_initializer='normal', activation='tanh'))
model.add(Dense(1, kernel_initializer='normal'))

# Compiling the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Fitting the ANN to the Training set
history = model.fit(X_train, y_train ,batch_size = 10, validation_data = (X_test, y_test), epochs = 15, verbose=1)

#save the model
model.save(r'../models/MSFT_ann.h5')

#save the feature data normalizer
scalerfilename_x = f'../normalizers/scaler_x_MSFT_ann.pkl'
pickle.dump(scaler_x, open(scalerfilename_x, 'wb'))


# #save the output normaliser
scalerfilename_y = f'../normalizers/scaler_y_MSFT_ann.pkl'
pickle.dump(scaler_y, open(scalerfilename_y, 'wb'))

Epoch 1/15
8/8 [==============================] - 0s 17ms/step - loss: 0.4347 - val_loss: 0.3272
Epoch 2/15
1/8 [==>...........................] - ETA: 0s - loss: 0.5540

2022-07-15 02:23:59.685029: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-15 02:23:59.845301: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


8/8 [==============================] - 0s 7ms/step - loss: 0.4120 - val_loss: 0.3050
Epoch 3/15
8/8 [==============================] - 0s 6ms/step - loss: 0.3785 - val_loss: 0.2705
Epoch 4/15
8/8 [==============================] - 0s 6ms/step - loss: 0.3240 - val_loss: 0.2165
Epoch 5/15
8/8 [==============================] - 0s 6ms/step - loss: 0.2370 - val_loss: 0.1426
Epoch 6/15
8/8 [==============================] - 0s 6ms/step - loss: 0.1349 - val_loss: 0.0718
Epoch 7/15
8/8 [==============================] - 0s 6ms/step - loss: 0.0542 - val_loss: 0.0361
Epoch 8/15
8/8 [==============================] - 0s 6ms/step - loss: 0.0227 - val_loss: 0.0272
Epoch 9/15
8/8 [==============================] - 0s 6ms/step - loss: 0.0171 - val_loss: 0.0190
Epoch 10/15
8/8 [==============================] - 0s 6ms/step - loss: 0.0118 - val_loss: 0.0120
Epoch 11/15
8/8 [==============================] - 0s 6ms/step - loss: 0.0077 - val_loss: 0.0098
Epoch 12/15
8/8 [==============================] 

# Training LSTM model for AAPL

In [71]:
X = np.asarray(apple[indicators], np.float32)
Y = np.asarray(apple['Close'], np.float32)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

y_train = y_train.reshape(-1,1)
mms_X = MinMaxScaler()
mms_y = MinMaxScaler()

scaler_x = mms_X.fit(X_train)
scaler_y = mms_y.fit(y_train)

X_train = scaler_x.transform(X_train)
y_train = scaler_y.transform(y_train)

X_test = scaler_x.transform(X_test)
y_test = y_test.reshape(-1,1)
y_test = scaler_y.transform(y_test)

pca = PCA(n_components=4)

X_train= pca.fit_transform(X_train)
X_test = pca.transform(X_test)

X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)

print(X_test[0].shape)

# Defining ANN neural network
model = Sequential()
model.add(LSTM(units=50, input_shape=(1,4), activation='relu'))
model.add(Dense(units=50,activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))

# Compiling the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Fitting the ANN to the Training set
history = model.fit(X_train, y_train ,batch_size = 10, validation_data = (X_test, y_test), epochs = 15, verbose=1)

#save the model
model.save(r'../models/AAPL_lstm.h5')

(1, 4)
Epoch 1/15
1/8 [==>...........................] - ETA: 3s - loss: 0.0765

2022-07-15 03:04:01.446309: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


8/8 [==============================] - 1s 36ms/step - loss: 0.0892 - val_loss: 0.0790
Epoch 2/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0815 - val_loss: 0.0720
Epoch 3/15
1/8 [==>...........................] - ETA: 0s - loss: 0.0990

2022-07-15 03:04:01.822706: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


8/8 [==============================] - 0s 14ms/step - loss: 0.0740 - val_loss: 0.0638
Epoch 4/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0652 - val_loss: 0.0548
Epoch 5/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0551 - val_loss: 0.0453
Epoch 6/15
8/8 [==============================] - 0s 12ms/step - loss: 0.0451 - val_loss: 0.0351
Epoch 7/15
8/8 [==============================] - 0s 12ms/step - loss: 0.0348 - val_loss: 0.0252
Epoch 8/15
8/8 [==============================] - 0s 12ms/step - loss: 0.0247 - val_loss: 0.0167
Epoch 9/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0165 - val_loss: 0.0103
Epoch 10/15
8/8 [==============================] - 0s 12ms/step - loss: 0.0106 - val_loss: 0.0060
Epoch 11/15
8/8 [==============================] - 0s 12ms/step - loss: 0.0068 - val_loss: 0.0037
Epoch 12/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0049 - val_loss: 0.0027
Epoch 13/15
8/8 [=====================

# Training LSTM model for GOOG

In [72]:
X = np.asarray(google[indicators], np.float32)
Y = np.asarray(google['Close'], np.float32)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

y_train = y_train.reshape(-1,1)
mms_X = MinMaxScaler()
mms_y = MinMaxScaler()

scaler_x = mms_X.fit(X_train)
scaler_y = mms_y.fit(y_train)

X_train = scaler_x.transform(X_train)
y_train = scaler_y.transform(y_train)

X_test = scaler_x.transform(X_test)
y_test = y_test.reshape(-1,1)
y_test = scaler_y.transform(y_test)

pca = PCA(n_components=4)

X_train= pca.fit_transform(X_train)
X_test = pca.transform(X_test)

X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)

print(X_test[0].shape)

# Defining ANN neural network
model = Sequential()
model.add(LSTM(units=50, input_shape=(1,4), activation='relu'))
model.add(Dense(units=50,activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))

# Compiling the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Fitting the ANN to the Training set
history = model.fit(X_train, y_train ,batch_size = 10, validation_data = (X_test, y_test), epochs = 15, verbose=1)

#save the model
model.save(r'../models/GOOG_lstm.h5')

(1, 4)
Epoch 1/15
1/8 [==>...........................] - ETA: 3s - loss: 0.0542

2022-07-15 03:04:51.651000: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


8/8 [==============================] - 1s 37ms/step - loss: 0.0620 - val_loss: 0.0509
Epoch 2/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0575 - val_loss: 0.0466
Epoch 3/15
1/8 [==>...........................] - ETA: 0s - loss: 0.0639

2022-07-15 03:04:52.038851: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


8/8 [==============================] - 0s 13ms/step - loss: 0.0532 - val_loss: 0.0421
Epoch 4/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0488 - val_loss: 0.0370
Epoch 5/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0436 - val_loss: 0.0317
Epoch 6/15
8/8 [==============================] - 0s 12ms/step - loss: 0.0380 - val_loss: 0.0261
Epoch 7/15
8/8 [==============================] - 0s 12ms/step - loss: 0.0321 - val_loss: 0.0206
Epoch 8/15
8/8 [==============================] - 0s 12ms/step - loss: 0.0264 - val_loss: 0.0155
Epoch 9/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0210 - val_loss: 0.0114
Epoch 10/15
8/8 [==============================] - 0s 12ms/step - loss: 0.0166 - val_loss: 0.0084
Epoch 11/15
8/8 [==============================] - 0s 12ms/step - loss: 0.0136 - val_loss: 0.0065
Epoch 12/15
8/8 [==============================] - 0s 12ms/step - loss: 0.0110 - val_loss: 0.0055
Epoch 13/15
8/8 [=====================

# Training LSTM model for TSLA

In [74]:
X = np.asarray(tesla[indicators], np.float32)
Y = np.asarray(tesla['Close'], np.float32)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

y_train = y_train.reshape(-1,1)
mms_X = MinMaxScaler()
mms_y = MinMaxScaler()

scaler_x = mms_X.fit(X_train)
scaler_y = mms_y.fit(y_train)

X_train = scaler_x.transform(X_train)
y_train = scaler_y.transform(y_train)

X_test = scaler_x.transform(X_test)
y_test = y_test.reshape(-1,1)
y_test = scaler_y.transform(y_test)

pca = PCA(n_components=4)

X_train= pca.fit_transform(X_train)
X_test = pca.transform(X_test)

X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)

print(X_test[0].shape)

# Defining ANN neural network
model = Sequential()
model.add(LSTM(units=50, input_shape=(1,4), activation='relu'))
model.add(Dense(units=50,activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))

# Compiling the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Fitting the ANN to the Training set
history = model.fit(X_train, y_train ,batch_size = 10, validation_data = (X_test, y_test), epochs = 15, verbose=1)

#save the model
model.save(r'../models/TSLA_lstm.h5')

(1, 4)
Epoch 1/15
1/8 [==>...........................] - ETA: 3s - loss: 0.0718

2022-07-15 03:05:42.111326: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


8/8 [==============================] - 1s 37ms/step - loss: 0.0944 - val_loss: 0.1005
Epoch 2/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0883 - val_loss: 0.0949
Epoch 3/15
1/8 [==>...........................] - ETA: 0s - loss: 0.0965

2022-07-15 03:05:42.506625: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


8/8 [==============================] - 0s 14ms/step - loss: 0.0839 - val_loss: 0.0898
Epoch 4/15
8/8 [==============================] - 0s 12ms/step - loss: 0.0790 - val_loss: 0.0843
Epoch 5/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0732 - val_loss: 0.0779
Epoch 6/15
8/8 [==============================] - 0s 12ms/step - loss: 0.0666 - val_loss: 0.0703
Epoch 7/15
8/8 [==============================] - 0s 12ms/step - loss: 0.0589 - val_loss: 0.0619
Epoch 8/15
8/8 [==============================] - 0s 12ms/step - loss: 0.0505 - val_loss: 0.0528
Epoch 9/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0422 - val_loss: 0.0436
Epoch 10/15
8/8 [==============================] - 0s 12ms/step - loss: 0.0339 - val_loss: 0.0338
Epoch 11/15
8/8 [==============================] - 0s 12ms/step - loss: 0.0255 - val_loss: 0.0245
Epoch 12/15
8/8 [==============================] - 0s 12ms/step - loss: 0.0184 - val_loss: 0.0163
Epoch 13/15
8/8 [=====================

# Training LSTM model for MSFT

In [76]:
X = np.asarray(microsoft[indicators], np.float32)
Y = np.asarray(microsoft['Close'], np.float32)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

y_train = y_train.reshape(-1,1)
mms_X = MinMaxScaler()
mms_y = MinMaxScaler()

scaler_x = mms_X.fit(X_train)
scaler_y = mms_y.fit(y_train)

X_train = scaler_x.transform(X_train)
y_train = scaler_y.transform(y_train)

X_test = scaler_x.transform(X_test)
y_test = y_test.reshape(-1,1)
y_test = scaler_y.transform(y_test)

pca = PCA(n_components=4)

X_train= pca.fit_transform(X_train)
X_test = pca.transform(X_test)

X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)

print(X_test[0].shape)

# Defining ANN neural network
model = Sequential()
model.add(LSTM(units=50, input_shape=(1,4), activation='relu'))
model.add(Dense(units=50,activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))

# Compiling the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Fitting the ANN to the Training set
history = model.fit(X_train, y_train ,batch_size = 10, validation_data = (X_test, y_test), epochs = 15, verbose=1)

#save the model
model.save(r'../models/MSFT_lstm.h5')

(1, 4)
Epoch 1/15
1/8 [==>...........................] - ETA: 3s - loss: 0.1000

2022-07-15 03:06:41.397522: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


8/8 [==============================] - 1s 37ms/step - loss: 0.0900 - val_loss: 0.0970
Epoch 2/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0837 - val_loss: 0.0904
Epoch 3/15
1/8 [==>...........................] - ETA: 0s - loss: 0.0491

2022-07-15 03:06:41.793402: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


8/8 [==============================] - 0s 14ms/step - loss: 0.0781 - val_loss: 0.0835
Epoch 4/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0717 - val_loss: 0.0756
Epoch 5/15
8/8 [==============================] - 0s 14ms/step - loss: 0.0643 - val_loss: 0.0664
Epoch 6/15
8/8 [==============================] - 0s 12ms/step - loss: 0.0560 - val_loss: 0.0559
Epoch 7/15
8/8 [==============================] - 0s 12ms/step - loss: 0.0470 - val_loss: 0.0444
Epoch 8/15
8/8 [==============================] - 0s 12ms/step - loss: 0.0372 - val_loss: 0.0332
Epoch 9/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0281 - val_loss: 0.0233
Epoch 10/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0204 - val_loss: 0.0153
Epoch 11/15
8/8 [==============================] - 0s 12ms/step - loss: 0.0147 - val_loss: 0.0093
Epoch 12/15
8/8 [==============================] - 0s 12ms/step - loss: 0.0103 - val_loss: 0.0058
Epoch 13/15
8/8 [=====================

# Training GRU model for AAPL

In [79]:
X = np.asarray(apple[indicators], np.float32)
Y = np.asarray(apple['Close'], np.float32)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

y_train = y_train.reshape(-1,1)
mms_X = MinMaxScaler()
mms_y = MinMaxScaler()

scaler_x = mms_X.fit(X_train)
scaler_y = mms_y.fit(y_train)

X_train = scaler_x.transform(X_train)
y_train = scaler_y.transform(y_train)

X_test = scaler_x.transform(X_test)
y_test = y_test.reshape(-1,1)
y_test = scaler_y.transform(y_test)

pca = PCA(n_components=4)

X_train= pca.fit_transform(X_train)
X_test = pca.transform(X_test)

X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)

print(X_test[0].shape)

# Defining ANN neural network
model = Sequential()
model.add(GRU(units=50, input_shape=(1,4), activation='relu'))
model.add(Dense(units=50,activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))

# Compiling the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Fitting the ANN to the Training set
history = model.fit(X_train, y_train ,batch_size = 10, validation_data = (X_test, y_test), epochs = 15, verbose=1)

#save the model
model.save(r'../models/AAPL_gru.h5')

(1, 4)
Epoch 1/15


2022-07-15 04:02:25.526728: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


8/8 [==============================] - 1s 44ms/step - loss: 0.0764 - val_loss: 0.0886
Epoch 2/15
8/8 [==============================] - 0s 16ms/step - loss: 0.0635 - val_loss: 0.0729
Epoch 3/15


2022-07-15 04:02:26.115560: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


8/8 [==============================] - 0s 14ms/step - loss: 0.0507 - val_loss: 0.0573
Epoch 4/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0381 - val_loss: 0.0425
Epoch 5/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0272 - val_loss: 0.0283
Epoch 6/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0176 - val_loss: 0.0170
Epoch 7/15
8/8 [==============================] - 0s 14ms/step - loss: 0.0105 - val_loss: 0.0094
Epoch 8/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0062 - val_loss: 0.0053
Epoch 9/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0043 - val_loss: 0.0034
Epoch 10/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0038 - val_loss: 0.0027
Epoch 11/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0035 - val_loss: 0.0025
Epoch 12/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0034 - val_loss: 0.0024
Epoch 13/15
8/8 [=====================

# Training GRU model for GOOG

In [80]:
X = np.asarray(google[indicators], np.float32)
Y = np.asarray(google['Close'], np.float32)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

y_train = y_train.reshape(-1,1)
mms_X = MinMaxScaler()
mms_y = MinMaxScaler()

scaler_x = mms_X.fit(X_train)
scaler_y = mms_y.fit(y_train)

X_train = scaler_x.transform(X_train)
y_train = scaler_y.transform(y_train)

X_test = scaler_x.transform(X_test)
y_test = y_test.reshape(-1,1)
y_test = scaler_y.transform(y_test)

pca = PCA(n_components=4)

X_train= pca.fit_transform(X_train)
X_test = pca.transform(X_test)

X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)

print(X_test[0].shape)

# Defining ANN neural network
model = Sequential()
model.add(GRU(units=50, input_shape=(1,4), activation='relu'))
model.add(Dense(units=50,activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))

# Compiling the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Fitting the ANN to the Training set
history = model.fit(X_train, y_train ,batch_size = 10, validation_data = (X_test, y_test), epochs = 15, verbose=1)

#save the model
model.save(r'../models/GOOG_gru.h5')

(1, 4)
Epoch 1/15


2022-07-15 04:03:35.654852: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


8/8 [==============================] - 1s 40ms/step - loss: 0.0586 - val_loss: 0.0598
Epoch 2/15
8/8 [==============================] - 0s 16ms/step - loss: 0.0517 - val_loss: 0.0506
Epoch 3/15
1/8 [==>...........................] - ETA: 0s - loss: 0.0790

2022-07-15 04:03:36.106830: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


8/8 [==============================] - 0s 15ms/step - loss: 0.0453 - val_loss: 0.0421
Epoch 4/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0390 - val_loss: 0.0337
Epoch 5/15
8/8 [==============================] - 0s 15ms/step - loss: 0.0332 - val_loss: 0.0260
Epoch 6/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0270 - val_loss: 0.0198
Epoch 7/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0221 - val_loss: 0.0149
Epoch 8/15
8/8 [==============================] - 0s 14ms/step - loss: 0.0170 - val_loss: 0.0118
Epoch 9/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0137 - val_loss: 0.0095
Epoch 10/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0107 - val_loss: 0.0082
Epoch 11/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0084 - val_loss: 0.0075
Epoch 12/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0070 - val_loss: 0.0070
Epoch 13/15
8/8 [=====================

# Training GRU model for TSLA

In [81]:
X = np.asarray(tesla[indicators], np.float32)
Y = np.asarray(tesla['Close'], np.float32)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

y_train = y_train.reshape(-1,1)
mms_X = MinMaxScaler()
mms_y = MinMaxScaler()

scaler_x = mms_X.fit(X_train)
scaler_y = mms_y.fit(y_train)

X_train = scaler_x.transform(X_train)
y_train = scaler_y.transform(y_train)

X_test = scaler_x.transform(X_test)
y_test = y_test.reshape(-1,1)
y_test = scaler_y.transform(y_test)

pca = PCA(n_components=4)

X_train= pca.fit_transform(X_train)
X_test = pca.transform(X_test)

X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)

print(X_test[0].shape)

# Defining ANN neural network
model = Sequential()
model.add(GRU(units=50, input_shape=(1,4), activation='relu'))
model.add(Dense(units=50,activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))

# Compiling the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Fitting the ANN to the Training set
history = model.fit(X_train, y_train ,batch_size = 10, validation_data = (X_test, y_test), epochs = 15, verbose=1)

#save the model
model.save(r'../models/TSLA_gru.h5')

(1, 4)
Epoch 1/15


2022-07-15 04:04:24.361552: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


8/8 [==============================] - 1s 43ms/step - loss: 0.0940 - val_loss: 0.1018
Epoch 2/15
8/8 [==============================] - 0s 16ms/step - loss: 0.0812 - val_loss: 0.0899
Epoch 3/15
1/8 [==>...........................] - ETA: 0s - loss: 0.0925

2022-07-15 04:04:24.815582: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


8/8 [==============================] - 0s 15ms/step - loss: 0.0714 - val_loss: 0.0789
Epoch 4/15
8/8 [==============================] - 0s 14ms/step - loss: 0.0612 - val_loss: 0.0677
Epoch 5/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0519 - val_loss: 0.0554
Epoch 6/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0415 - val_loss: 0.0430
Epoch 7/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0320 - val_loss: 0.0319
Epoch 8/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0232 - val_loss: 0.0230
Epoch 9/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0162 - val_loss: 0.0161
Epoch 10/15
8/8 [==============================] - 0s 14ms/step - loss: 0.0113 - val_loss: 0.0110
Epoch 11/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0075 - val_loss: 0.0080
Epoch 12/15
8/8 [==============================] - 0s 14ms/step - loss: 0.0054 - val_loss: 0.0062
Epoch 13/15
8/8 [=====================

# Training GRU model for MSFT

In [82]:
X = np.asarray(microsoft[indicators], np.float32)
Y = np.asarray(microsoft['Close'], np.float32)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

y_train = y_train.reshape(-1,1)
mms_X = MinMaxScaler()
mms_y = MinMaxScaler()

scaler_x = mms_X.fit(X_train)
scaler_y = mms_y.fit(y_train)

X_train = scaler_x.transform(X_train)
y_train = scaler_y.transform(y_train)

X_test = scaler_x.transform(X_test)
y_test = y_test.reshape(-1,1)
y_test = scaler_y.transform(y_test)

pca = PCA(n_components=4)

X_train= pca.fit_transform(X_train)
X_test = pca.transform(X_test)

X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)

print(X_test[0].shape)

# Defining ANN neural network
model = Sequential()
model.add(GRU(units=50, input_shape=(1,4), activation='relu'))
model.add(Dense(units=50,activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))

# Compiling the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Fitting the ANN to the Training set
history = model.fit(X_train, y_train ,batch_size = 10, validation_data = (X_test, y_test), epochs = 15, verbose=1)

#save the model
model.save(r'../models/MSFT_gru.h5')

(1, 4)
Epoch 1/15


2022-07-15 04:05:42.387812: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


8/8 [==============================] - 1s 41ms/step - loss: 0.0945 - val_loss: 0.0792
Epoch 2/15
8/8 [==============================] - 0s 15ms/step - loss: 0.0812 - val_loss: 0.0681
Epoch 3/15
1/8 [==>...........................] - ETA: 0s - loss: 0.0743

2022-07-15 04:05:42.830489: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


8/8 [==============================] - 0s 15ms/step - loss: 0.0703 - val_loss: 0.0573
Epoch 4/15
8/8 [==============================] - 0s 14ms/step - loss: 0.0590 - val_loss: 0.0467
Epoch 5/15
8/8 [==============================] - 0s 14ms/step - loss: 0.0483 - val_loss: 0.0363
Epoch 6/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0376 - val_loss: 0.0269
Epoch 7/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0290 - val_loss: 0.0184
Epoch 8/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0211 - val_loss: 0.0122
Epoch 9/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0157 - val_loss: 0.0077
Epoch 10/15
8/8 [==============================] - 0s 14ms/step - loss: 0.0113 - val_loss: 0.0050
Epoch 11/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0082 - val_loss: 0.0038
Epoch 12/15
8/8 [==============================] - 0s 13ms/step - loss: 0.0064 - val_loss: 0.0036
Epoch 13/15
8/8 [=====================

# Random Forest Regressor for AAPL

In [ ]:
X = np.asarray(apple[indicators], np.float32)
Y = np.asarray(apple['Close'], np.float32)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

y_train = y_train.reshape(-1,1)
mms_X = MinMaxScaler()
mms_y = MinMaxScaler()

scaler_x = mms_X.fit(X_train)
scaler_y = mms_y.fit(y_train)

X_train = scaler_x.transform(X_train)
y_train = scaler_y.transform(y_train)

X_test = scaler_x.transform(X_test)
y_test = y_test.reshape(-1,1)
y_test = scaler_y.transform(y_test)

pca = PCA(n_components=4)

X_train= pca.fit_transform(X_train)
X_test = pca.transform(X_test)

X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)

print(X_test[0].shape)

# Defining ANN neural network
model = Sequential()
model.add(GRU(units=50, input_shape=(1,4), activation='relu'))
model.add(Dense(units=50,activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))

# Compiling the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Fitting the ANN to the Training set
history = model.fit(X_train, y_train ,batch_size = 10, validation_data = (X_test, y_test), epochs = 15, verbose=1)

#save the model
model.save(r'../models/AAPL_gru.h5')